In [1]:
!pip install -q langchain langchain-community langchain-huggingface transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is in

In [2]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [3]:
import os
from huggingface_hub import login
from getpass import getpass

# Try to read token from environment variable
HF_TOKEN = os.getenv("HF_token") or os.getenv("HUGGINGFACE_HUB_TOKEN")

if HF_TOKEN:
    try:
        login(token=HF_TOKEN)
        print("Successfully logged in to Hugging Face using environment variable.")
    except Exception as e:
        print(f"Failed to login using environment token: {e}")
else:
    # Ask user to enter token securely
    print("Please enter your Hugging Face token:")
    HF_TOKEN = getpass("Hugging Face token: ").strip()
    if HF_TOKEN:
        os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
        try:
            login(token=HF_TOKEN)
            print("Successfully logged in to Hugging Face.")
        except Exception as e:
            print(f"Login failed: {e}")
    else:
        print("No token provided. Some models may not be accessible.")

Please enter your Hugging Face token:


Hugging Face token:  ········


Successfully logged in to Hugging Face.


In [4]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os

os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')

2025-07-16 17:34:41.588107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752687281.950928      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752687282.058175      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Set your Hugging Face model name here
MODEL_NAME = "google/gemma-2-2b-it"  
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [6]:
# Fix 2: Update the LLM setup to use proper chat template
def setup_llm(model_name="google/gemma-2-2b-it"):
    """Setup the Hugging Face LLM pipeline with proper chat template"""
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        token=True,
    )
    
    # Create pipeline with proper settings
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        return_full_text=False,  # This is key - only return the generated part
    )
    
    return HuggingFacePipeline(pipeline=pipe)

In [7]:
# ============= DOCUMENT PROCESSING =============
def load_and_process_pdf(pdf_path):
    """Load PDF and split into chunks"""
    # Load PDF
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks = text_splitter.split_documents(documents)
    print(f"Created {len(chunks)} chunks from the PDF")

    return chunks

In [8]:
# ============= EMBEDDINGS AND VECTOR STORE =============
def create_vector_store(chunks):
    """Create embeddings and vector store"""
    # Initialize embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
    )


    # Create vector store
    vectorstore = FAISS.from_documents(chunks, embeddings)
    print("Vector store created successfully")

    return vectorstore

In [9]:
def create_prompt_template_with_memory():
    return PromptTemplate(
        input_variables=["context", "chat_history", "question"],
        template="""<bos><start_of_turn>user
You are an expert research assistant to a university professor. You have access to research documents and should help with academic inquiries in a scholarly and professional manner.
Your role:
- Provide accurate, well-researched responses based on the provided context
- Cite relevant information from the documents when applicable
- Maintain an academic tone while being helpful and clear
- If you don't know something, admit it rather than speculating
- Help with research analysis, literature review, and academic writing

Context from research documents:
{context}

Previous conversation:
{chat_history}

Current question: {question}
<end_of_turn>
<start_of_turn>model
""")

In [10]:
#Fix 2: Simple memory management functions
def format_chat_history(conversation_history):
    """Format conversation history for the prompt"""
    if not conversation_history:
        return "No previous conversation."
    
    formatted = []
    for i in range(0, len(conversation_history), 2):
        if i + 1 < len(conversation_history):
            formatted.append(f"Human: {conversation_history[i]}")
            formatted.append(f"Assistant: {conversation_history[i + 1]}")
    
    return "\n".join(formatted)

def add_to_memory(conversation_history, question, answer, max_exchanges=6):
    """Add question-answer pair to memory and manage size"""
    conversation_history.append(question)
    conversation_history.append(answer)
    
    # Keep only last max_exchanges (each exchange = 2 items: question + answer)
    max_items = max_exchanges * 2
    if len(conversation_history) > max_items:
        conversation_history[:] = conversation_history[-max_items:]
    
    return conversation_history

def clear_memory(conversation_history):
    """Clear conversation memory"""
    conversation_history.clear()
    print("Memory cleared!")

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableMap


# Fix 3: Create RAG chain with memory support
def create_rag_chain_with_memory(pdf_path, model_name=MODEL_NAME):
    """Create RAG chain that supports memory"""
    
    print("Setting up RAG with Memory...")
    
    # Setup components
    llm = setup_llm(model_name)
    chunks = load_and_process_pdf(pdf_path)
    vectorstore = create_vector_store(chunks)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
    # Create prompt template
    prompt_template = create_prompt_template_with_memory()
    
    def format_docs(docs):
        """Format documents for context"""
        return "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])
    
    # Create the chain
    qa_chain = (
        RunnableMap({
            "question": lambda x: x["question"],
            "chat_history": lambda x: x["chat_history"],
            "context": lambda x: format_docs(retriever.get_relevant_documents(x["question"]))
        })
        | prompt_template  
        | llm              
        | StrOutputParser()
    )
    
    print("RAG chain with memory ready!")
    return qa_chain

In [12]:
# Fix 4: Main chat function
def ask_question(qa_chain, conversation_history, question):
    """Ask a question with memory support"""
    
    # Format chat history
    chat_history = format_chat_history(conversation_history)
    
    # Generate response
    response = qa_chain.invoke({
        "question": question,
        "chat_history": chat_history
    })
    
    # Add to memory
    add_to_memory(conversation_history, question, response)
    
    return response

# Fix 5: Utility functions
def show_memory(conversation_history):
    """Show current conversation history"""
    print("=== CONVERSATION HISTORY ===")
    formatted = format_chat_history(conversation_history)
    print(formatted)
    print("==========================")

def get_memory_size(conversation_history):
    """Get number of exchanges in memory"""
    return len(conversation_history) // 2

In [13]:
# Usage Example:
print("=== Testing RAG with Memory - Simple Functions ===")

# Step 1: Create the RAG chain
qa_chain = create_rag_chain_with_memory(
    pdf_path="/kaggle/input/sample-v3/attention_is_all_you_need.pdf",
    model_name=MODEL_NAME
)

# Step 2: Initialize conversation history (empty list)
conversation_history = []

# Step 3: Ask questions
print("\n--- starting the conversation ---")
question0 = "hello how you doing today?"
response0 = ask_question(qa_chain, conversation_history, question0)
print(f"Q: {question0}")
print(f"A: {response0}")

print("\n--- First Question ---")
question1 = "What is the difference between attention and RNN?"
response1 = ask_question(qa_chain, conversation_history, question1)
print(f"Q: {question1}")
print(f"A: {response1}")

print("\n--- Second Question (with memory) ---")
question2 = "Which is better?"
response2 = ask_question(qa_chain, conversation_history, question2)
print(f"Q: {question2}")
print(f"A: {response2}")

print("\n--- Third Question ---")
question3 = "Can you give me more details about the better one?"
response3 = ask_question(qa_chain, conversation_history, question3)
print(f"Q: {question3}")
print(f"A: {response3}")

# Step 4: Show memory
show_memory(conversation_history)

=== Testing RAG with Memory - Simple Functions ===
Setting up RAG with Memory...


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Q: hello how you doing today?
A: I am doing well, thank you. How can I assist you today? 


--- First Question ---


skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)


Q: Which is better?
A: It's not as simple as saying one is "better" – both RNNs and Transformer models have strengths and weaknesses, making them suitable for different tasks. 

Here's a breakdown of the key differences you asked about:

**RNNs:**

* **Strengths:** RNNs excel at handling sequential data, where the order of input is crucial. They are great for tasks like language modeling, text generation, and speech recognition where understanding context over time is essential. 
* **Weaknesses:** RNNs can struggle with long sequences due to their sequential nature. The "memory cell" can lose information after multiple steps, making them less effective at capturing long-term dependencies.

**Transformers (specifically the Transformer architecture used in the mentioned documents):**

* **Strengths:**  Transformers are specifically designed to tackle long-range dependencies in text. Self-attention allows the model to consider all the words in a sentence simultaneously, capturing relation

skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)


Q: Can you give me more details about the better one?
A: You're asking about a very interesting question! It's not just about which is "better", but rather the best model for a specific task. 

Here's a breakdown of why Transformers are generally considered superior for tasks involving long-range dependencies, and why RNNs have a different place in the field:

**Transformers: The Power of Attention**

The key to Transformer's success is its **self-attention mechanism**.  Imagine you're reading a sentence. You don't just focus on the words one by one, but you also consider how each word relates to others.  Transformers capture this complex relationship in a single step. 

Think about these key points:

* **Parallel Processing:** Transformers can process all words in a sentence simultaneously, unlike RNNs that process sequentially. This allows them to capture long-range dependencies that RNNs might miss. 
* **Contextual Understanding:** Self-attention enables the model to understand the 